In [71]:
import requests
import json
import pandas as pd
import re
from bs4 import BeautifulSoup
import csv

In [88]:
#github job portal scraping using requests

# creating a list store the responses of each source url based on pagination
num = 1
res_li =[]
response = ''
while response != []:
    url = "https://jobs.github.com/positions.json?description=all&page=" + str(num)
    response = requests.request("GET", url)
    response = json.loads(response.text)
    res_li.append(response)
    num += 1

# populating the list based on the job source, position, company, names, location, etc.
github_information = []
for sub_list in res_li:
    for item in sub_list:
#         print(item)
        source = 'Github Jobs'
        comp_name = item['company']
        role = item['title']
        desp = item ['description']
        clean = re.compile('<.*?>')
        desp = re.sub(clean, '', desp)
        link = item ['url']
        loc = item['location']
        job_id = item['id']
        job_type = item['type']
        date = item['created_at']
        how_to_apply = item['how_to_apply']
        clean = re.compile('<.*?>')
        how_to_apply = re.sub(clean, '', how_to_apply)
        github_information.append((source, comp_name, role, desp, link, loc, job_id, job_type, date, how_to_apply))
print(github_information[0])

# storing the data in a dataframe
df1 = pd.DataFrame(github_information)

('Github Jobs', 'Zappos.com', 'Senior Software Engineer- Full Stack', 'Work remotely temporarily due to COVID-19\nWe are a small team helping groups throughout the organization solve problems and develop new ideas. Our projects focus on the future of Zappos by building interactive experiences centered around customer service. We are a nimble team that moves fast and empowers others to build and experiment with new Zappos experiences rapidly.\nWe are looking for candidates who have a sense of ownership to deliver quick but extensible solutions. You will have the freedom to incubate new ideas with the backing of a large yet passionate-about-culture organization. Our current platform is built on Ruby on Rails and React.js, but we will always prioritize finding the right tool for the job.\nSome of our recent projects include launching ZapposKids.com (https://www.zappos.com/e/shops/kids) and Zappos Goods for Good (https://www.zappos.com/e/goods-for-good).\nWhat you\'ll be doing:\nResponsibi

In [91]:
#Indeed job portal scraping using requests

url = "https://indeed-indeed.p.rapidapi.com/apisearch"

querystring = {"q":"all","v":"2","format":"json","sort":"relevance","radius":"","l":"worldwide","st":"jobsite","callback":"test","limit":"","co":"in","useragent":"testnull","fromage":"test","latlong":"test","filter":"test","jt":"test","start":"test","highlight":"test","chnl":"test","userip":"test","publisher":"9801269362146606"}

headers = {
    'x-rapidapi-host': "indeed-indeed.p.rapidapi.com",
    'x-rapidapi-key': " "
    }

response = requests.request("GET", url, headers=headers, params=querystring)
response = json.loads(response.text[5:-1])

indeed_information = []
for item in response['results']:
#     print(item)
    source  = 'Indeed'
    job_type = item['source']
    comp_name = item['company']
    role = item['jobtitle']
    desp = item ['snippet']
    link = item ['url']
    loc = item['city']
    date = item['date']
    job_id = item['jobkey']
    posted_since = item['formattedRelativeTime']
    indeed_information.append((source, comp_name, role, desp, link, loc, date, job_id, posted_since))
print(indeed_information[0])

# storing the data in a dataframe
df2 = pd.DataFrame(indeed_information)

('Indeed', 'Agriculture Insurance Company of India Limited (AIC of India)', '[Government] AIC of India 2020', 'Agriculture Insurance Company of India Limited (AIC of India). Invites applications from eligible candidates for the post of Multi Media Specialist on…', 'https://www.indeed.co.in/viewjob?jk=878d33a4eb8e58c2&qd=fykWmKlmRr18BBf5vVtcrALGxPTkYhXe2xFPsjPn4pF4ZCXVAlfoAL676zUEpmB5V8znEYRk2ElzFWPoA17JFTx4DKknYZtQmHs5MdPwabU&indpubnum=9801269362146606&chnl=test&atk=1ec89tealrhro800', '', 'Sat, 06 Jun 2020 10:59:26 GMT', '878d33a4eb8e58c2', '26 days ago')


In [74]:
# Monster job portal scrapping using BeautifulSoup

url = 'https://www.monsterindia.com/search/freshers-jobs?searchId=115b51ed-0edd-457d-be02-c3ec675b2f68'
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
response = requests.get(url, headers = agent)
response = response.text

soup = BeautifulSoup(response, 'lxml')
soup.prettify()

main = soup.find_all('div', class_ = 'job-tittle')
main

# scraping and populating the job roles
jr_li = []
for elem in main:
#     print(elem.h3.a.text)
    job = elem.h3.a.text
    jr_li.append(job)
print("JOB ROLES---{}".format(jr_li))
print()

# scraping and populating the company names
comp = soup.find_all('span', class_ = 'company-name')
comp_li = []
for elem in comp:
    if elem.a == None:
        elem = 'NA'
    else:
        name = elem.a.text
    comp_li.append(name)
print("COMPANY NAME---{}".format(comp_li))
print()

# scraping and populating the job location
loc = soup.find_all('div', class_ = 'col-xxs-12 col-sm-5 text-ellipsis')
loc_li = []
for elem in loc:
    if elem == ' ':
        continue
    else:
        elem = elem.small.text.strip().replace(',','').replace('\n','').replace(" ","")
        loc_li.append(elem)
print("LOCATION---{}".format(loc_li))
print()

# scraping and populating the job experience required
exp = soup.find_all('div', class_ = 'exp col-xxs-12 col-sm-3 text-ellipsis')
exp_li = []
for elem in exp:
    elem = elem.small.text.strip()
    exp_li.append(elem)
# filling the empty cells to make the length of the arrays/lists equal as other lists
exp_li.insert(19, "0-1 years")
exp_li.insert(20, "0-2 years")
exp_li.insert(21, "0-2 years")
exp_li.insert(22, "0-2 years")
print("EXPERIENCE---{}".format(exp_li))
print()

# scraping and populating the salary package offered
sal = soup.find_all('div', class_ = 'package col-xxs-12 col-sm-4 text-ellipsis')
sal_li = []
for elem in sal:
    elem = elem.small.text.strip()
    sal_li.append(elem)
print("SALARY--{}".format(sal_li))
print()


JOB ROLES---['Medical coding Jobs  - Nursing, Staff Nurse, DGNM Freshers', 'URGENT REQUIREMENT FOR MALE PHARMACIST (FRESHERS) FOR A NEWLY OPENING COMPANY COCHIN', 'Medical coding Jobs  - Pharmacist, B Pharm, M Pharm, Pharm D Freshers', 'Freshers Can Call @ 9779924404. We Are Hiring Graduates, also 10+2 pass for BPO Salary 12k to 20k/ M', 'Airport Jobs | Airlines Jobs for Freshers 12th, Graduates Can Apply!', 'Medical Coding Jobs For Physiotherapist, Physiotherapy, BPT, MPT & Dentist, BDS, MDS Freshers', 'Medical coding Jobs - Biotechnology, Biomedical, Biochemistry Freshers', 'Day job : Hiring Freshers/ 12th Pass/ Under graduate/ Graduate in Day Shifts work  Saahil 6362028414', 'Medical coding Jobs  - Zoology, Microbiology, MLT Freshers', 'Freshers job in sap ', 'freshers job in bandhan bank', 'Freshers in logistics ', 'Graduates / Freshers / BCom / BBA / BBM / Accounts / Finance / Insurance / Collection / Customer Sup', 'FRESHERS JOBS - IMMEDIATELY WANTED HR EXECUTIVES -', 'freshers  

In [93]:
github = df1.rename(columns={0: 'Source', 1: 'Company', 2: 'Job Role', 3: 'Description', 4: 'Link', 5: 'Location', 6: 'Job_id', 7: 'Job type', 8: 'Date Posted', 9: 'How to apply'})
github['Experinece Required'] = "--"
github['Package Offered'] = "--"
github['Posted since'] = "--"

#displaying first 2 rows of the github job portal
github.head(2)

,Source,Company,Job Role,Description,Link,Location,Job_id,Job type,Date Posted,How to apply,Experinece Required,Package Offered,Posted since
0,Github Jobs,Zappos.com,Senior Software Engineer- Full Stack,Work remotely temporarily due to COVID-19\nWe ...,https://jobs.github.com/positions/911a5212-129...,USA Remote,911a5212-129b-491f-95d4-f7eaa19bdefa,Full Time,Thu Jul 02 17:01:53 UTC 2020,Please click the link to apply!\nhttps://app.j...,--,--,--
1,Github Jobs,Virtual Pricing Director,Back End Engineer: Node.js | data | GraphQL,Virtual Pricing Director is hiring a Back End ...,https://jobs.github.com/positions/22098e20-4b3...,UK,22098e20-4b3e-48fb-b109-d6dd72bddd70,Full Time,Wed Jul 01 15:05:54 UTC 2020,"We would be pleased to chat and tell you more,...",--,--,--


In [94]:
indeed = df2.rename(columns={0: 'Source', 1: 'Company', 2: 'Job Role', 3: 'Description', 4: 'Link', 5: 'Location', 6:'Date Posted', 7: 'Job_id', 8: 'Posted since'})
indeed['Experinece Required'] = "--"
indeed['Package Offered'] = "--"
indeed['How to apply'] = "--"
indeed['Job type'] = job_type

# displaying the first 2 rows of the indeed job portal
indeed.head(2)

,Source,Company,Job Role,Description,Link,Location,Date Posted,Job_id,Posted since,Experinece Required,Package Offered,How to apply,Job type
0,Indeed,Agriculture Insurance Company of India Limited...,[Government] AIC of India 2020,Agriculture Insurance Company of India Limited...,https://www.indeed.co.in/viewjob?jk=878d33a4eb...,,"Sat, 06 Jun 2020 10:59:26 GMT",878d33a4eb8e58c2,26 days ago,--,--,--,Freshers Home
1,Indeed,SAZ India,Application For Ground Stafff Fresher,Invite The Application For Ground Staff. Good ...,https://www.indeed.co.in/viewjob?jk=b725e670a2...,Imphal,"Fri, 29 May 2020 05:59:25 GMT",b725e670a2985470,30+ days ago,--,--,--,Freshers Home


In [95]:
monster = pd.DataFrame({'Source': 'Monster', 'Company': comp_li, 'Job Role': jr_li,'Location': loc_li, 'Experience Required':exp_li, 'Package offered':sal_li  })
monster["Description"] = "--"
monster["Link"] = "--"
monster["Date Posted"] = "--"
monster["Job_id"] = "--"
monster["Package offered"] = "--"
monster["How to apply"] = "--"
monster["Posted since"] = "--"
monster["Job type"] = "--"

# displaying the first 2 rows of the monster job portal
monster.head(2)

,Source,Company,Job Role,Location,Experience Required,Package offered,Description,Link,Date Posted,Job_id,How to apply,Posted since,Job type
0,Monster,I Skills Solutions,"Medical coding Jobs - Nursing, Staff Nurse, D...",Chennai,0-1 years,--,--,--,--,--,--,--,--
1,Monster,International Trade Links,URGENT REQUIREMENT FOR MALE PHARMACIST (FRESHE...,Cochin/Kochi/Ernakulam,0-1 years,--,--,--,--,--,--,--,--


In [96]:
# concatinating all three data frames into one
compile_data = [github, indeed, monster]
master_data = pd.concat(compile_data)

# displaying the first 2 rows of master data jobs
master_data.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,Company,Date Posted,Description,Experience Required,Experinece Required,How to apply,Job Role,Job type,Job_id,Link,Location,Package Offered,Package offered,Posted since,Source
0,Zappos.com,Thu Jul 02 17:01:53 UTC 2020,Work remotely temporarily due to COVID-19\nWe ...,NaN,--,Please click the link to apply!\nhttps://app.j...,Senior Software Engineer- Full Stack,Full Time,911a5212-129b-491f-95d4-f7eaa19bdefa,https://jobs.github.com/positions/911a5212-129...,USA Remote,--,NaN,--,Github Jobs
1,Virtual Pricing Director,Wed Jul 01 15:05:54 UTC 2020,Virtual Pricing Director is hiring a Back End ...,NaN,--,"We would be pleased to chat and tell you more,...",Back End Engineer: Node.js | data | GraphQL,Full Time,22098e20-4b3e-48fb-b109-d6dd72bddd70,https://jobs.github.com/positions/22098e20-4b3...,UK,--,NaN,--,Github Jobs


In [79]:
# generating a csv 
output = master_data.to_csv('master_data_for_jobs.csv', index=False)